# 📊 Data Analysis – Người 3

Phân tích dữ liệu bao gồm:
- Thống kê mô tả mở rộng
- Phân phối dữ liệu (Histogram, KDE)
- Kiểm tra phân phối chuẩn (Skewness, Kurtosis)
- Phân tích tương quan
- Phân tích pattern & trend theo thời gian
- Phân tích theo City & Restaurant
- Phân tích Customer & Payment
- Tổng hợp insights & đề xuất


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
plt.style.use('ggplot')

df = pd.read_csv('../data/foodpanda_orders.csv')
df.head()

## 📌 Thống kê mô tả mở rộng

In [ ]:
desc = df.describe().T
desc['median'] = df.median(numeric_only=True)
desc['mode'] = df.mode().iloc[0]
desc['skewness'] = df.skew(numeric_only=True)
desc['kurtosis'] = df.kurtosis(numeric_only=True)
desc

## 📌 Phân phối dữ liệu (Histogram + KDE)

In [ ]:
numeric_cols = df.select_dtypes(include=['int64','float64']).columns

for col in numeric_cols:
    plt.figure(figsize=(7,4))
    sns.histplot(df[col], kde=True)
    plt.title(f'Phân phối: {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.show()

## 📌 Kiểm tra phân phối chuẩn

In [ ]:
for col in numeric_cols:
    print(f'Biến: {col}')
    print(f'  Skewness  : {df[col].skew():.3f}')
    print(f'  Kurtosis  : {df[col].kurtosis():.3f}\n')

## 📌 Phân tích tương quan giữa các biến

In [ ]:
plt.figure(figsize=(8,6))
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap Tương Quan')
plt.show()

## 📌 Pattern & Trend theo thời gian

In [ ]:
if 'order_time' in df.columns:
    df['order_time'] = pd.to_datetime(df['order_time'])
    df['date'] = df['order_time'].dt.date
    df['hour'] = df['order_time'].dt.hour

    daily = df.groupby('date').size()
    hourly = df.groupby('hour').size()

    plt.figure(figsize=(10,4))
    daily.plot()
    plt.title('Xu hướng số đơn theo ngày')
    plt.show()

    plt.figure(figsize=(10,4))
    hourly.plot(kind='bar')
    plt.title('Pattern số đơn theo giờ')
    plt.show()

## 📌 Phân tích theo City & Restaurant

In [ ]:
if 'city' in df.columns:
    print(df['city'].value_counts())

if 'restaurant' in df.columns:
    print(df['restaurant'].value_counts().head(10))

## 📌 Phân tích Customer & Payment

In [ ]:
if 'customer_id' in df.columns:
    print('Số lượng khách hàng khác nhau:', df['customer_id'].nunique())
    print(df['customer_id'].value_counts().head())

if 'payment_method' in df.columns:
    print(df['payment_method'].value_counts())

## 📌 Insights & Đề xuất
- Nếu skewness cao → dữ liệu lệch phải, cần log-transform.
- Nếu thời gian giờ cao điểm rõ rệt → tối ưu nhân sự giao hàng.
- Thành phố/restaurant có đơn cao → ưu tiên marketing.
- Nếu khách hàng lặp lại nhiều → triển khai loyalty.
- Nếu payment có xu hướng COD → cần cải thiện thanh toán online.
